In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set()

In [ ]:
def date_parser(m):
    eng_months = {'januari' : 'January',
                 'februari' : 'February',
                 'mars' : 'March',
                 'april' : 'April',
                 'maj' : 'May',
                 'juni' : 'June',
                 'juli' : 'July',
                 'augusti' : 'August',
                 'september' : 'September',
                 'oktober' : 'October',
                 'november' : 'November',
                 'december' : 'December'}
    
    eng = eng_months[m]
    
    return pd.datetime.strptime(eng,'%B').date().strftime('%B')
    
    
death_df = pd.read_csv('scb_dead_per_month.csv',encoding='ISO-8859-1')

In [ ]:
death_df['månad'] = death_df['månad'].apply(date_parser)
death_df['månad'] = pd.Categorical(death_df['månad'],['January','February','March','April','May','June',
                                         'July','August','September','October','November','December'])

In [ ]:
death_df

In [ ]:
death_df = death_df.groupby('månad').sum()

death_df

In [ ]:
scb_pop = pd.read_csv('scb_population.csv',encoding='ISO-8859-1')

scb_pop

In [ ]:
pd.set_option("display.precision", 2)
deaths_per_m = death_df / (scb_pop.loc[0,:] / 1e6)
deaths_per_m = deaths_per_m.T
deaths_per_m

In [ ]:
monthly_deaths = []

for r,v in deaths_per_m.iterrows():
    monthly_deaths.extend(v)
    


In [ ]:
dates = pd.date_range('1990-01-01','2019-12-31',freq='M')
dates

In [ ]:
monthly_timeline = pd.DataFrame(data=monthly_deaths,index=dates,columns=['dead_per_million'])

In [ ]:
curr_pop = 10327589 #from scb.se for 2019-12-31

scb_2020 = pd.read_csv('scb-dead_per_day.csv',sep=';')
scb_2020 = scb_2020.groupby('Månad').sum()
scb_2020 = scb_2020['2020']
scb_2020 = pd.DataFrame(scb_2020)
scb_2020.reset_index(inplace=True)
scb_2020['Månad'] = pd.Categorical(scb_2020['Månad'],['januari','februari','mars','april','maj','juni',
                                                     'juli','augusti','september','oktober','november','december'])

scb_2020.sort_values('Månad',inplace=True)
scb_2020.index = pd.date_range('2020-01-01','2020-12-31',freq='M')
scb_2020 = scb_2020.loc[:'2020-05-31']
scb_2020['2020'] = scb_2020['2020'] / (curr_pop/1e6)
scb_2020.rename(columns={'2020' : 'dead_per_million'},inplace=True)
scb_2020

In [ ]:
monthly_timeline

In [ ]:
monthly_timeline.plot(figsize=(18,12))
scb_2020['dead_per_million'].plot(color='r')

In [ ]:
monthly_timeline = pd.concat([monthly_timeline['dead_per_million'],scb_2020['dead_per_million']],axis=0)
monthly_timeline

In [ ]:
monthly_timeline = pd.DataFrame(monthly_timeline.sort_values(ascending=False))
monthly_timeline['rank'] = range(1,len(monthly_timeline) + 1)
monthly_timeline.head(50)

In [ ]:
mean = monthly_timeline.mean()
mean

In [ ]:
monthly_timeline.loc['2020-05-31']

In [ ]:
top_20 = pd.DataFrame(monthly_timeline[:20])
top_20

In [ ]:
colors = ['b'] * 18
colors.append('r')
colors.append('b')
print (colors)
top_20.index = top_20.index.date
top_20['dead_per_million'].plot(kind='bar',
                                figsize=(18,12),
                                color=colors,
                                title='SWEDEN - Top-20 deadliest months January 1990 - May 2020\nDataSource: SCB.se')
plt.ylabel('deaths per million')
plt.axhline(mean['dead_per_million'],color='orange',ls='dashed',label='average 1990-2019')
plt.legend(loc='upper right')
plt.savefig('scb_population_deaths_1990-2020.jpg',format='jpg')